# Optimising Survey Data Processes

##### Setting Up and Importing Essential Modules

In [1]:
#pip install pdf2image pathlib regex pytesseract pillow pandas fuzzywuzzy 

In [2]:
# pip install more_itertools

In [3]:
# pip install pyenchant

In [2]:
#importing modules needed
import re
import enchant
from fuzzywuzzy import process
import pandas as pd

C:\Users\shekh\anaconda\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Phase 1: Data Extraction

The first pdf we take for data extraction (Balance Sheet 13.pdf) does not have clear defined tables and the information can not be directly read by the computer, we use the following "pdf_to_image" and "read_text_from_scanned_document" functions that returns all the information extacted in the form of a string using the pytesseract module for Optical Character Recognition.

In [3]:
#function for converting pdf to images
def pdf_to_image(pdfs):
    from pdf2image import convert_from_path
    from pathlib import Path
    pages = convert_from_path(pdfs, 350) #first converting all the pages from the pdf to images
    i = 1
    folder=[]
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"  
        page.save(image_name, "JPEG")             #saves it in the current working directory
        folder.append(str(Path.cwd())+"\\"+image_name) #folder is a list of strings with the path all the image files created for future reference
        i = i+1
    return folder

#function for scanning text from the image
def read_text_from_scanned_document(file): #inputs a folder of documents path
    import pytesseract
    from PIL import Image
    text = "\n"
    for image_path in file:
        image = Image.open(image_path)
        text += pytesseract.image_to_string(image)+"\nNext Page\n"
    return text  

In [4]:
fl = pdf_to_image(r"Balance Sheet (13)-redacted.pdf")
txt = read_text_from_scanned_document(fl)

In [12]:
print(txt[0:70])


STANDALONE BALANCE SHEET AS AT 31 MARCH, 2022 _

ASSETS
NON-CURRENT A


## Phase 2: Data Cleaning

The forthcoming lines of code encompasses the workflow of converting the above raw text into a tabular format so that we get information about the following attributes: [Item Name, CY, PY, Note, Identifier]

In [50]:
# Create a regular expression pattern to match the word followed by anything (.*)
pattern = r'NOTES TO'
# Use re.search to find the first occurrence of the match word in the input string
match_txt = re.search(pattern, txt)
check_text = txt[0:match_txt.span()[0]]

In [51]:
#selecting words not in dictionary and does not have a numerical value
English = enchant.Dict("en_US")

def select_lines(text):
    matches = text.split("\n")  #splitting every line
    selected_lines=[]     #this will contain a list of every selected line as a list of words(list of lists)

    for line in matches:
        check=[]
        words = line.split(" ") #splitting every word in a line
        if "" not in words:
            for word in words:
                if word.strip("()") != "":
                    if English.check(word.strip("()")) == False and word.rstrip("s") in English.suggest(word):
                        check.append(English.check(word.rstrip("s")))
                    else:
                        check.append(English.check(word.strip("()")))
                        
            if check.count(True)> len(words)*0.8 or (re.search(r'\d', line) and check.count(True)> len(words)*0.7) :#still contains lines that does not make sense but have a number in them
                        selected_lines.append(line)

    #Final list after splitting
    #splitting wherever there is either an Uppercase letter or a digit followed by a space 
    for row in selected_lines:
        selected_lines[selected_lines.index(row)] = re.split('(?<=\s)(?=[A-Z\d=\-\(])|-',str(row))
    return selected_lines
    
    

In [52]:
Selected_lines = select_lines(check_text) 

In [53]:
Selected_lines

[['STANDALONE ', 'BALANCE ', 'SHEET ', 'AS ', 'AT ', '31 ', 'MARCH, ', '2022'],
 ['Note ', 'As at ', 'As at'],
 ['No. ', '31 ', 'March, ', '2022 ', '31 ', 'March, ', '2021'],
 ['(Rupees in ', 'Lakhs) ', '(Rupees in ', 'Lakhs)'],
 ['ASSETS'],
 ['NON', 'CURRENT ', 'ASSETS'],
 ['a. ', 'Property, plant and equipment ', '4.1 ', '43,392.42 ', '44,295.63'],
 ['b. ', 'Capital work', 'in', 'progress ', '4.2 ', '1,392.10 ', '707.16'],
 ['c. ', 'Right of ', 'Use ', 'Assets ', '4.3 ', '16,464.17 ', '7,129.93'],
 ['d. ', 'Intangible assets ', '4.4 ', '578.99 ', '489.66'],
 ['e. ', 'Intangible assets under development ', '4.5 ', '89.75 ', '159.79'],
 ['f. ', 'Financial assets'],
 ['(i) ', 'Investment ', '5 ', '8,721.95 ', '8,721.95'],
 ['(ii) ', 'Other financial assets ', '6 ', '428.10 ', '608.81'],
 ['q. ', 'Deferred tax assets ', '(net) ', '26 ', '4,301.02 ', '1,531.83'],
 ['h. ', 'Income tax assets ', '(net) ', '7 ', '891.21 ', '1,187.85'],
 ['i. ', 'Other non', 'current assets ', '8 ', '7,184.12

In [54]:
def is_regex_match(pattern, input_string):
    match = re.search(pattern, input_string)  # Use re.search to find the regex pattern in the input string
    return bool(match)          # Return True if a match is found, otherwise False
    

def clean_lines(lines_selected,no_of_columns,identifiers,no_of_numeric_cols,threshold):
    cat = 0
    patrn = re.compile(r'^[\d\(|\-|=]')  #matches a decimal or an hyphen or equal sign
    for item in lines_selected[:]:
        item_string = " ".join(item)
        if len(item)<6:
            matched, score= process.extractOne(item_string,identifiers)
            if score>=threshold:
                cat = matched
        item.append(cat)
        #removing no
        if no_of_numeric_cols==3:
            if len(item)<no_of_columns-1 or ((not is_regex_match(patrn,item[-4])) or (not is_regex_match(patrn,item[-3])) or (not is_regex_match(patrn,item[-2]))):
                lines_selected.remove(item)
        elif no_of_numeric_cols==2:
            if len(item)<no_of_columns-1 or ((not is_regex_match(patrn,item[-3])) or (not is_regex_match(patrn,item[-2]))):
                lines_selected.remove(item)
            
    #correcting item names
    for item in lines_selected[:]:
        if len(item)!=no_of_columns and no_of_columns==6:
            item[1] = ''.join(item[1:-4])
            del item[2:-4]
        elif len(item)!=no_of_columns and no_of_columns==4:
            item[0] = ''.join(item[0:-3])
            del item[1:-3]
            
        if len(item)!=no_of_columns:
            lines_selected.remove(item)
                     
    return lines_selected
    

In [55]:
Identifiers=["Current Assets","Non Current Assets","Non Current Liabilities","Current liabilities","Equity","Revenue","Expenses","Tax","Loan","Income","Manufacturing and trading"]
selected_lines = clean_lines(Selected_lines,6,Identifiers,3,90)

In [56]:
selected_lines

[['a. ',
  'Property, plant and equipment ',
  '4.1 ',
  '43,392.42 ',
  '44,295.63',
  'Non Current Assets'],
 ['b. ',
  'Capital workinprogress ',
  '4.2 ',
  '1,392.10 ',
  '707.16',
  'Non Current Assets'],
 ['c. ',
  'Right of Use Assets ',
  '4.3 ',
  '16,464.17 ',
  '7,129.93',
  'Non Current Assets'],
 ['d. ',
  'Intangible assets ',
  '4.4 ',
  '578.99 ',
  '489.66',
  'Non Current Assets'],
 ['e. ',
  'Intangible assets under development ',
  '4.5 ',
  '89.75 ',
  '159.79',
  'Non Current Assets'],
 ['(i) ', 'Investment ', '5 ', '8,721.95 ', '8,721.95', 'Non Current Assets'],
 ['(ii) ',
  'Other financial assets ',
  '6 ',
  '428.10 ',
  '608.81',
  'Non Current Assets'],
 ['q. ',
  'Deferred tax assets (net) ',
  '26 ',
  '4,301.02 ',
  '1,531.83',
  'Non Current Assets'],
 ['h. ',
  'Income tax assets (net) ',
  '7 ',
  '891.21 ',
  '1,187.85',
  'Non Current Assets'],
 ['i. ',
  'Other noncurrent assets ',
  '8 ',
  '7,184.12 ',
  '2,426.02',
  'Non Current Assets'],
 ['a.

In [57]:
def conv_to_formatted_table(list_of_list,column_names): #column_names is list
    #convert to table:
    #format of the table has to be of six columns- s.no,item name, note number, current year, previous year, identifier
    df = pd.DataFrame(list_of_list,columns = column_names)
    
    #defining column types and removing abnormal data rows
    #Convert Current Year and Previous Year into float type
    if "Note no" in column_names:
        df["Note no"] = pd.to_numeric(df["Note no"], errors="coerce")
    df['Current Year'] = pd.to_numeric(df['Current Year'].str.replace(r'[,;)]', '', regex=True).replace(r'[=-]', '0', regex=True).replace('\(', "-", regex =  True), errors='coerce')
    df['Previous Year'] = pd.to_numeric(df['Previous Year'].str.replace(r'[,;)]', '', regex=True).replace(r'[=-]', '0', regex=True).replace('\(', "-", regex =  True), errors='coerce')


    # Delete rows where 'Values' column does not contain numeric values
    return df.dropna()

In [58]:
check_df = conv_to_formatted_table(selected_lines, ("Serial no.","Item Name","Note no","Current Year","Previous Year","Identifier"))

In [59]:
check_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36 entries, 0 to 38
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Serial no.     36 non-null     object 
 1   Item Name      36 non-null     object 
 2   Note no        36 non-null     float64
 3   Current Year   36 non-null     float64
 4   Previous Year  36 non-null     float64
 5   Identifier     36 non-null     object 
dtypes: float64(3), object(3)
memory usage: 2.0+ KB


In [60]:
check_df.head()

,Serial no.,Item Name,Note no,Current Year,Previous Year,Identifier
0,a.,"Property, plant and equipment",4.1,43392.42,44295.63,Non Current Assets
1,b.,Capital workinprogress,4.2,1392.10,707.16,Non Current Assets
2,c.,Right of Use Assets,4.3,16464.17,7129.93,Non Current Assets
3,d.,Intangible assets,4.4,578.99,489.66,Non Current Assets
4,e.,Intangible assets under development,4.5,89.75,159.79,Non Current Assets


In [61]:
#In order to map, we will create two subtables, one for balance sheet and the other for P&L
# Balance Sheet-> Current Assets, Current Liabilities
# P&L -> Income, Expenses

In [62]:
BSh_table = check_df[(check_df["Identifier"]== "Current Assets") | (check_df["Identifier"]== "Current liabilities")]
PL_table = check_df[(check_df["Identifier"]== "Income") | (check_df["Identifier"]== "Expenses") | (check_df["Identifier"]== "Revenue")]

In [63]:
#First for the BSh table
BSh_table["Identifier"].unique()

array(['Current Assets', 'Current liabilities'], dtype=object)

In [64]:
notes_to_search=list(BSh_table["Note no"].astype(int).astype("string")+" "+BSh_table["Item Name"])+(list(PL_table["Note no"].astype(int).astype("string")+" "+PL_table["Item Name"]))
print(notes_to_search)

['9 Inventories ', '10 Trade receivables ', '14 Cash and cash eaurvaients ', '12 Other Balances with Banks ', '14 Other financial assets ', '15 Other current assets ', '3 Total outstanding dues of creditors other than micro enterprises ', '18 Lease Liabilities ', '23 Other financial liabilities ', '24 Provisions ', '25 Other current liabilities ', '27 Revenue from operations ', '28 Other income ', '29 Changes in inventories of finished goods, ', '30 Employee benefits expense ', '32 Depreciation and amortisation expense ', '33 Other expenses ', '39 Remeasurement of the defined benefit ']


In [65]:
#Getting text from the notes
notes_text = txt[match_txt.span()[0]:]


In [66]:
#Finding notes like pattern from notes_text

pattern = r"^\d+[\s\w+]+"   #for matching notes; digit followed by words

notes_list = re.findall(pattern, notes_text, re.MULTILINE)  #Contains a lot of noise

In [67]:
# Cleaning and splitting the list
cleaned_list = [item.strip() for item in notes_list if item.strip()]
separated_list = [item.split("\n") for item in cleaned_list]

# Flatten the nested lists
flattened_list = [item for sublist in separated_list for item in sublist]

notes=[]
months=["January","February","March","April","May","June","July","August","September","October","November","December"]
for item in flattened_list:
    words = item.split(" ")
    if len(words)!=1:
        if "" not in words:
            check = [English.check(word.lower()) for word in words if word not in months ]   #contains bool values 
            #checking for the conditions
            if check.count(True)> len(words)*0.66: 
                notes.append(item)

In [68]:
print("notes extracted from the pdf:\n",notes)
print("\nnotes taken from  the tables\n:",notes_to_search)

notes extracted from the pdf:
 ['10 each', '6 Other non', '7 Non', '8 Other non', '9 Inventories', '10 Trade receivables', '11 Cash and cash equivalents', '12 Other Balances with Banks', '12 months', '13 Loans', '14 Other current financial assets', '15 Other current assets', '18 Lease Liabilities', '19 Other non current financial liabilities', '20 Non', '21 Other non', '22 Trade payables', '23 Other current financial liabilities', '24 Provisions', '25 Other current liabilities', '26 Deferred tax balances', '27 Revenue from operations', 'Revenue from contracts with customers', '28 Other income', 'NOTES TO THE STANDALONE FINANCIAL STATEMENTS FOR THE YEAR ENDED 31 MARCH', '33 Other expenses', '34 Income taxes', '35 Payment to auditors', 'Expenditure incurred for corporate social responsibility', '2021 Rs', 'Next Page', 'NOTES TO THE STANDALONE FINANCIAL STATEMENTS FOR THE YEAR ENDED 31 MARCH', '40 Related party disclosures', 'Earnings per share', 'NOTES TO THE STANDALONE FINANCIAL STATEME

In [69]:
#Defining a function for lose matching
def find_from_ref(Match_list,lookup_list):

    # Set the threshold for matching similarity
    threshold = 86

    # Perform fuzzy matching
    matched = []
    for item in Match_list:
        if item!= None:
            match, score= process.extractOne(item, lookup_list)
            if score >= threshold:
                matched.append((item, match))    
    return matched

In [70]:
new_identifiers = list(list(zip(*find_from_ref(notes_to_search,notes)))[1])

In [71]:
new_identifiers

['9 Inventories',
 '10 Trade receivables',
 '11 Cash and cash equivalents',
 '12 Other Balances with Banks',
 '14 Other current financial assets',
 '15 Other current assets',
 '6 Other non',
 '18 Lease Liabilities',
 '23 Other current financial liabilities',
 '24 Provisions',
 '25 Other current liabilities',
 '27 Revenue from operations',
 '28 Other income',
 '9 Inventories',
 '33 Other expenses',
 'NOTES TO THE STANDALONE FINANCIAL STATEMENTS FOR THE YEAR ENDED 31 MARCH']

In [72]:
len(new_identifiers)

16

In [73]:
len(notes_to_search)

18

In [74]:
#editing
new_identifiers.append("22 Trade payables")
new_identifiers.append("13 Loans")
new_identifiers.pop(6)
new_identifiers.pop(12)
new_identifiers.pop(13)

'NOTES TO THE STANDALONE FINANCIAL STATEMENTS FOR THE YEAR ENDED 31 MARCH'

In [75]:
#Cleaning
def find_lines_with_identifiers_list(text, identfiers_list):
    result = []
    for line in text.split('\n'):
        for identifier in identfiers_list:
            if line.startswith(identifier):
                rest_of_line = line[len(identifier):].strip()
                result.append(identifier+rest_of_line)
                break  # Once a match is found, move to the next line
    return result

In [76]:
new_identifiers_cleaned = find_lines_with_identifiers_list(notes_text,new_identifiers)
print(new_identifiers_cleaned)

['9 Inventories(lower of cost and net realisable value) (refer note 2.5)', '10 Trade receivables(unsecured)', '11 Cash and cash equivalents', '12 Other Balances with Banks', '13 Loans(unsecured, considered good)', '14 Other current financial assets(unsecured, considered good)', '15 Other current assets(unsecured, considered good)', '18 Lease Liabilities', '22 Trade payables', '23 Other current financial liabilities', '24 Provisions', '25 Other current liabilities', '27 Revenue from operations', '28 Other income', '33 Other expenses']


In [77]:
notes_selected_lines = select_lines(notes_text)
result = clean_lines(notes_selected_lines,4,new_identifiers_cleaned,2,90)
detail_table = conv_to_formatted_table(result[:175], ("Item Name","Current Year","Previous Year","Note"))

In [78]:
#As we have only mapped for the items which have notes in the balance sheet and P&L data,
#We are only focused on extracting the financial information
df_detail = detail_table[detail_table["Note"]!=0]  #table needed

In [79]:
df_detail.head()

,Item Name,Current Year,Previous Year,Note
61,a. Raw materials including packaging material ...,37234.08,17291.32,9 Inventories(lower of cost and net realisable...
62,Goodsintransit .,16573.16,11468.51,9 Inventories(lower of cost and net realisable...
64,c. Finished goods (other than those acquired f...,46232.15,35766.07,9 Inventories(lower of cost and net realisable...
65,Finished goodsintransit,5224.14,3922.63,9 Inventories(lower of cost and net realisable...
66,d. Stockintrade (in respect of goods acquired ...,6783.77,6041.79,9 Inventories(lower of cost and net realisable...


In [80]:
#Splitting note and note no

# Define regular expression patterns for numeric and text values
numeric_pattern = r'((\-|\d)+(\.\d+)?)\b'
text_pattern = r'([A-Za-z\s]+[^\d]*)'

# Extract numeric values using regular expression
numeric_values = df_detail['Note'].str.extract(numeric_pattern, expand=False)[0]  #extracts three groups, we are taking the first (overall) group

# Extract text values using regular expression
text_values = df_detail['Note'].str.extract(text_pattern, expand=False)

#after checking add numeric values and text values to the data frame
df_detail['Note no'] = numeric_values
df_detail["Notes"] = text_values
df_detail=df_detail.drop("Note",axis=1)


df_detail["Note no"] = df_detail["Note no"].astype(float)
check_df['Note no'] = check_df['Note no'].astype(float)
check_df= check_df.dropna()
df_detail= df_detail.dropna()

C:\Users\shekh\AppData\Local\Temp\ipykernel_17676\3356080489.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_detail['Note no'] = numeric_values
C:\Users\shekh\AppData\Local\Temp\ipykernel_17676\3356080489.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_detail["Notes"] = text_values


In [81]:
BSPL_merged_table = df_detail.merge(check_df[["Note no","Identifier"]],on="Note no", how="left")

In [82]:
BSPL_merged_table

,Item Name,Current Year,Previous Year,Note no,Notes,Identifier
0,a. Raw materials including packaging material ...,37234.08,17291.32,9.0,Inventories(lower of cost and net realisable ...,Current Assets
1,Goodsintransit .,16573.16,11468.51,9.0,Inventories(lower of cost and net realisable ...,Current Assets
2,c. Finished goods (other than those acquired f...,46232.15,35766.07,9.0,Inventories(lower of cost and net realisable ...,Current Assets
3,Finished goodsintransit,5224.14,3922.63,9.0,Inventories(lower of cost and net realisable ...,Current Assets
4,d. Stockintrade (in respect of goods acquired ...,6783.77,6041.79,9.0,Inventories(lower of cost and net realisable ...,Current Assets
...,...,...,...,...,...,...
81,n. Expenditure on CSR (refer note 36),579.46,572.06,33.0,Other expenses,Expenses
82,oO. Legal and professional expenses (refer not...,1910.77,1634.97,33.0,Other expenses,Expenses
83,"p. Loss on sale of property, plant and equipme...",1.66,0.66,33.0,Other expenses,Expenses
84,q. E waste expenditure (refer note 48),4468.89,2966.30,33.0,Other expenses,Expenses


In [84]:
#Add another column in the merged table for mapping reference account
mapping = {"Current Assets":"Current Asset (Asset Side)",
          "Current liabilities":"Current Liability",
          "Revenue":"P & L Account",
          "Income":"P & L Account",
          "Expenses":"P & L Account"}
BSPL_merged_table["Reference Account"] = BSPL_merged_table['Identifier'].map(mapping)

In [85]:
#Formatting the Item names column
# Split the text only if the pattern is matched, otherwise keep original text
import pandas as pd
def split_text(text):
    if '.' in text:
        return text.split('. ', 1)
    else:
        return [text, text]

# Apply the custom function to each row
split_values = BSPL_merged_table['Item Name'].apply(split_text)

In [86]:
split_data = pd.DataFrame(split_values.tolist(), columns=['column1', 'column2'])
BSPL_merged_table["Item Name"]= split_data["column2"]

In [87]:
BSPL_merged_table

,Item Name,Current Year,Previous Year,Note no,Notes,Identifier,Reference Account
0,Raw materials including packaging material and...,37234.08,17291.32,9.0,Inventories(lower of cost and net realisable ...,Current Assets,Current Asset (Asset Side)
1,,16573.16,11468.51,9.0,Inventories(lower of cost and net realisable ...,Current Assets,Current Asset (Asset Side)
2,Finished goods (other than those acquired for ...,46232.15,35766.07,9.0,Inventories(lower of cost and net realisable ...,Current Assets,Current Asset (Asset Side)
3,Finished goodsintransit,5224.14,3922.63,9.0,Inventories(lower of cost and net realisable ...,Current Assets,Current Asset (Asset Side)
4,Stockintrade (in respect of goods acquired for...,6783.77,6041.79,9.0,Inventories(lower of cost and net realisable ...,Current Assets,Current Asset (Asset Side)
...,...,...,...,...,...,...,...
81,Expenditure on CSR (refer note 36),579.46,572.06,33.0,Other expenses,Expenses,P & L Account
82,Legal and professional expenses (refer note 35),1910.77,1634.97,33.0,Other expenses,Expenses,P & L Account
83,"Loss on sale of property, plant and equipment ...",1.66,0.66,33.0,Other expenses,Expenses,P & L Account
84,E waste expenditure (refer note 48),4468.89,2966.30,33.0,Other expenses,Expenses,P & L Account


In [88]:
#saving merged_table to excel  
BSPL_merged_table.to_excel("BSPL table.xlsx", index=False, header=True)

## PHASE 3:  Model Building

In [15]:
#Importing modules

# Data Processing
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [47]:
#importing data
ref = pd.read_excel("Ready references full.xlsx", sheet_name="Formatted")
solved = pd.read_excel("Solution file.xlsx",sheet_name="Both")
main_df = pd.read_excel("BSPL table.xlsx")

In [17]:
ref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 989 entries, 0 to 988
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Item Name          988 non-null    object
 1   Reference Account  905 non-null    object
 2   Block              899 non-null    object
 3   Item               862 non-null    object
 4   Block Item Name    911 non-null    object
dtypes: object(5)
memory usage: 38.8+ KB


In [18]:
#Preprocessing of references
ref["Item Name"]= ref["Item Name"].astype("string")
ref=ref[ref["Block"]!="C"] #Fixed assets are to be mapped separately
ref.dropna(inplace=True)  
ref["Block Item"]= ref["Block"].astype(str)+ref["Item"].astype(str)
references = ref[["Item Name","Reference Account","Block Item"]]

In [89]:
#Text normalisation
def normalise_text_from_col(df):
    #nltk.download('wordnet')

    # Convert text to lowercase
    df = df.str.lower()
       
    # Remove stopwords using NLTK
    stop_words = set(stopwords.words('english'))
    df = df.apply(lambda text: ' '.join([word for word in text.split() if word not in stop_words]))

    # Lemmatization using NLTK
    lemmatizer = WordNetLemmatizer()
    df = df.apply(lambda text: ' '.join([lemmatizer.lemmatize(word) for word in text.split()]))

    return df

In [20]:
#Feature extraction: convert words to vectors
def vectorise_column(df,text_column):
    import pandas as pd
    from sklearn.feature_extraction.text import TfidfVectorizer

    # Initialize TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer()

    # Fit and transform the text_column using TF-IDF
    tfidf_features = tfidf_vectorizer.fit_transform(df[text_column])

    # Convert the resulting sparse matrix to a pandas DataFrame
    tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

    return tfidf_df


In [90]:
tokenised_text = normalise_text_from_col(references["Item Name"])
references["Item Name"]=tokenised_text
vectorised_item_name = vectorise_column(pd.DataFrame(tokenised_text),"Item Name")

In [22]:
def reduce_dim(vectorised_data,dim):
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler

    # Standardize the data before applying PCA
    scaler = StandardScaler()
    standardized_vectorized_item_name = scaler.fit_transform(vectorised_data)

    # Apply PCA
    n_components = dim  # Choose the number of components you want to retain
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(standardized_vectorized_item_name)

    # Convert PCA result to a dataframe
    pca_df = pd.DataFrame(data=pca_result)
    return pca_df

In [23]:
def preprocessing_features(df):
    tk_text = normalise_text_from_col(df["Item Name"])
    df["Item Name"]=tk_text
    
    #vectorising
    vectorised_item = vectorise_column(pd.DataFrame(tk_text),"Item Name")
    #encoding
    encoded_data = pd.get_dummies(df, columns = ['Reference Account'])
    
    #cleaning
    if len(df.columns)==3:
        encoded_data.drop(columns=["Item Name","Block Item"],inplace=True)
    else:
        encoded_data.drop(columns=["Item Name"],inplace=True)
    encoded_data.reset_index(drop=True,inplace=True)   
    vectorised_item.reset_index(drop=True,inplace=True)
    
    #Dimension reduction
    reduced_item_name = reduce_dim(vectorised_item,68)
    reduced_ra = reduce_dim(encoded_data,3)
    
    #join
    combined_features_set = pd.concat([reduced_ra, reduced_item_name], axis=1)
    
    return combined_features_set

In [91]:
#Now fit the model(random forests)
# Split the data into features (X) and target (y) (Encoded)
y_raw = references["Block Item"]
label_encoder = LabelEncoder()
X = preprocessing_features(references[["Item Name","Reference Account"]])
y = label_encoder.fit_transform(y_raw)

C:\Users\shekh\AppData\Local\Temp\ipykernel_17676\1293405353.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Item Name"]=tk_text


In [38]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=1)

In [39]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [40]:
# Predict labels for the test data
predictions = rf.predict(X_test)

In [41]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7214285714285714


In [42]:
# We see an initial accuracy of 69%, let's improve upon this by training the model again
solved["Item"]=solved["Item"].fillna("")
updated_df = pd.concat([references,solved[["Item Name","Reference Account","Block Item"]]],ignore_index=True)
updated_df

,Item Name,Reference Account,Block Item
0,accessory used raw material,Manufacturing & Trading Account,D1
1,chemical,Manufacturing & Trading Account,D1
2,component used raw material,Manufacturing & Trading Account,D1
3,construction material,Manufacturing & Trading Account,D1
4,good trading (resale),Manufacturing & Trading Account,D1
...,...,...,...
692,GST payable,Current Liability,D14
693,Employees,Current Liability,D14
694,Advance against insurance claim,Current Liability,Out of ASI
695,Statutory remittances,Current Liability,D14


In [92]:
features = preprocessing_features(updated_df)
#Now fit the model(random forests)
# Split the data into features (X) and target (y) (Encoded)
y_raw = updated_df["Block Item"]
label_encoder = LabelEncoder()

X = features
y = label_encoder.fit_transform(y_raw)


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=6)

In [46]:
#Random forest
rf2 = RandomForestClassifier(n_estimators=100, random_state=42)
rf2.fit(X_train, y_train)

# Predict labels for the test data
predictions = rf2.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.75


In [93]:
X_new = main_df[["Item Name","Reference Account"]].dropna()
#find classes for the new values
main_X= preprocessing_features(X_new)
cat =  rf2.predict(main_X)
block_item  =label_encoder.inverse_transform(cat)
X_new["Block Item"]= block_item

In [33]:
final = main_df.join(X_new["Block Item"])
final.dropna()

,Item Name,Current Year,Previous Year,Note no,Notes,Identifier,Reference Account,Block Item
0,Raw materials including packaging material and...,37234.08,17291.32,9,Inventories(lower of cost and net realisable ...,Current Assets,Current Asset (Asset Side),D10
2,Finished goods (other than those acquired for ...,46232.15,35766.07,9,Inventories(lower of cost and net realisable ...,Current Assets,Current Asset (Asset Side),D10
3,Finished goodsintransit,5224.14,3922.63,9,Inventories(lower of cost and net realisable ...,Current Assets,Current Asset (Asset Side),D10
4,Stockintrade (in respect of goods acquired for...,6783.77,6041.79,9,Inventories(lower of cost and net realisable ...,Current Assets,Current Asset (Asset Side),D10
5,Stores and spares,9161.44,9855.30,9,Inventories(lower of cost and net realisable ...,Current Assets,Current Asset (Asset Side),D10
...,...,...,...,...,...,...,...,...
81,Expenditure on CSR (refer note 36),579.46,572.06,33,Other expenses,Expenses,P & L Account,F3
82,Legal and professional expenses (refer note 35),1910.77,1634.97,33,Other expenses,Expenses,P & L Account,F3
83,"Loss on sale of property, plant and equipment ...",1.66,0.66,33,Other expenses,Expenses,P & L Account,G2
84,E waste expenditure (refer note 48),4468.89,2966.30,33,Other expenses,Expenses,P & L Account,F3
